# SDS 2. Exam assignment
In this assignment we have to produce an analysis using a combination of Natural Language Processing (NLP) and Network Analysis techniques. Our analysis will try and look upon horror movie reviews in the IMDB dataset.

A logical hypothesis could thus be around halloween given it just have parsed and often is a subject in horror movies. This just ended up taking to much time to run, therefore a known horror movie was selected in 'A Nightmare on Elm Street':

***What is most central in reviews regarding the 'A Nightmare on Elm Street' movie and what is the sentiment regarding this movie?***

We are given 3 topics and some sub-elements that we need to answer while solving for this hypothesis This being:


*   Data processing with LLM's
*   Network analysis
*   Text classification

Each part is given a section below in this jupyter notebook.





# 1. Data Processing with LLMs
We have to do the foolowing steps:

- Implement local or cloud-based Large Language Models (LLMs) to:
  - Extract and structure relevant market data
  - Identify network relationships between entities
  - Perform named entity recognition and extraction
  - Transform unstructured text into analyzable formats


**Code Block 1**

  The first lines of code below is used to install necessary packages into collab. Transformers is used for languagemoddels, datasets to import datasets, pandas for datamanipulation, matplotlib for visualization and so on.

In [ ]:
# We start by installing necessary dependencies:
!pip install transformers datasets pandas spacy ollama networkx matplotlib tqdm -q
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Code block 2 and 3**

The next two codeblocks are used to create a connection to ollama which makes it possible to promt LLM to extract structured marketdata.

The code just below is used to install ollama, where the code below that is used to start the ollama server, which is used for the communication with the LLM.

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 2s (199 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [ ]:
# Required packages
import os
import threading
import subprocess
import json
import ollama
import networkx as nx
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from transformers import pipeline
import pandas as pd

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()


**Code block 4**

This is the LLM that we chose to use since it was the best at the moment given its various scores explained in the lectures. The model will as described above be used to extract marketdata and help with text manipulation.

In [ ]:
!ollama pull qwen2.5:14b

**Code block 5**

This code firstly import the package for downloading datasets and the imports it. Given its the IMDB dataset we have decided to work with this is the one downloaded. The dataset comes from Huggingface and have reviews on different movies. After importing it we look upon the data and see that we have a training set, test set and unsupervised with everything in it. We can therefor if we want to work with upto 50000 reviews. This will though require a lot of processing power to work with.

In [ ]:
from datasets import load_dataset
data = load_dataset("imdb")
data

**Code block 6**

This code first selects the training set and later shows the output. Since we in the assignment want an analyzable format this format neets to be changed.

In [ ]:
train_data = data['train']
train_data[[0]]

**Code block 7**

The dataset is transformed into a dataframe using the pandas package, to be able to easy filter the dataset. We could filter it based on movie titles or synonyms often used in horror movies, this would just give us a big dataset and require a lot of time when prompting the LLM. Therefore a movie is selected to minimize the time spent waiting.

In [ ]:
train_data = pd.DataFrame(train_data)
#horror_movies = ['The Exorcist', 'Psycho', 'Halloween', 'A Nightmare on Elm Street', 'The Shining', 'Scream']
horror_movies = ['A Nightmare on Elm Street']
horror_reviews = train_data[train_data['text'].str.contains('|'.join(horror_movies), case=False, na=False)]
print(horror_reviews.head())

**Code block 8**

This gives us a dataset for horror_reviews of 34. Eg 34 reviews.

In [ ]:
horror_reviews.size

**Code block 9**

This code is used to test the connection to the LLM by gennerating a prompt and showing the result. It can here be seen that there seems to be a valid connection.

In [ ]:
response = ollama.generate(model="qwen2.5:14b", prompt="Extract market-related information from the text: example text")
print(response)

**Code block 10**

Here we create a function that allows us to extract the market related information from each review. The data is then returned to the defined structured_text.

In [ ]:
def extract_market_data(text):
    response = ollama.generate(model="qwen2.5:14b", prompt=f"Extract market-related information from the text: {text}")

    if 'response' in response:
        structured_text = response['response']
        return structured_text
    else:
        return {"error": "Unexpected response format", "response": response}

**Code block 11**

This code is used to create an example to test whether our code works. Eg that we get the response as a structured format.

In [ ]:
sample_text = horror_reviews['text'].iloc[0]
print(extract_market_data(sample_text))

**Code block 12**

Since the format should be structured and easy to work with. The json format is chosen. We then get the output which shows what rows that include the horror movie, aswell as the review text and the structured_json format that is returned by the prompt.

In [ ]:
horror_reviews['structured_json'] = horror_reviews['text'].apply(extract_market_data)
print(horror_reviews[['text', 'structured_json']].head())

**Code block 13**

This code uses the spacy package to find named entities, such as persons or places from the reviews and saves it in a new coloumn named entities_json. This is done to later allow us to make network analysis on the data since places and persons can be connected to eachother.

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities
horror_reviews['entities_json'] = horror_reviews['text'].apply(extract_entities)
print(horror_reviews[['text', 'entities_json']].head())

# 2. Network Analysis

- Design and construct meaningful networks from the extracted data
- Implement bi-partite network analysis and corresponding projections
- Calculate and interpret key network metrics:
  - Various centrality measures
  - Network structure indicators
  - Community detection (if applicable)
- Provide clear interpretation of network analysis results

**Code block 14**

This code constructs a network graph using entities from a dataset of horror reviews, calculates centrality measures, and identifies the top entities based on their centrality.

In [ ]:
import networkx as nx
G = nx.Graph()
for _, row in horror_reviews.iterrows():
    entities = row['entities_json']
    for i, entity1 in enumerate(entities):
        for j, entity2 in enumerate(entities):
            if i < j:
                G.add_edge(entity1[0], entity2[0])

centrality = nx.degree_centrality(G)
print("Top entities by centrality:", sorted(centrality.items(), key=lambda x: x[1], reverse=True)[:10])

The entities with greatest importance for centrality is "Elm Street", "two" and "Nightmare", these keywords represtent connected titles in the network and as expected are connected to the title of the movie. Futhermore "Wes Craven's" has a high centrality which makes sence since the film are directed/produced by him.

The scores shows there is not a extreme influence of one keyword, but more spread influence.

**Code block 15**

This code creates a bipartite graph that connects the horror genre to associated entities (like characters, locations, concepts or other) extracted from a collection of horror reviews. Each horror node is linked to the various entities mentioned within the reviews. After building this network, the code then generates a projection focused solely on the enitities. In this projection, entities er connected if they occur more than once in the same review, with edge wheights indicating the frequency of this. Therefore it allows us to analyse entity relationships within the horrer genre based on prominent entities and their connections.

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
from collections import defaultdict
B = nx.Graph()
genre_node = "Horror"
B.add_node(genre_node, bipartite=0)
co_occurrence_counts = defaultdict(int)
for _, row in horror_reviews.iterrows():
    entities = row['entities_json']
    unique_entities = set(entities)
    for entity in unique_entities:
        B.add_node(entity, bipartite=1)
        B.add_edge(genre_node, entity)
    for entity1 in unique_entities:
        for entity2 in unique_entities:
            if entity1 != entity2:
                co_occurrence_counts[(entity1, entity2)] += 1
entity_projection = nx.Graph()
for (entity1, entity2), count in co_occurrence_counts.items():
    if count > 1:
        entity_projection.add_edge(entity1, entity2, weight=count)
print("Number of nodes in entity projection:", entity_projection.number_of_nodes())
print("Number of edges in entity projection:", entity_projection.number_of_edges())
weighted_centrality = nx.degree_centrality(entity_projection)
print("Top entities by weighted centrality:", sorted(weighted_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

The metrics of nodes and edges show a relative dense network with multiple connections between entities. This is done to help identify central themes within the horror narative in the movie.

- **Number of nodes: 34** – This indicates that the total number of unique entities identified in the reviews are 34. Each node represents a specific entity associated with the horror genre through the reviews of "A Nightmare on Elm Street."
- **Number of edges: 102** – The number of edges is the count of connections between different entities. An edge between two entities signifies a co-occurence in the same review.  
- **Top entities** – Lastly the top entities based on weighted centrality is found. Here it can be notessed that Elm street, the producer and persons in general is very central for the reviews.

**Code block 16**

This code analyzes which entities play a key role in the network.

In [ ]:
degree_centrality = nx.degree_centrality(entity_projection)
print("Top entities by degree centrality:", sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10])
betweenness_centrality = nx.betweenness_centrality(entity_projection)
print("Top entities by betweenness centrality:", sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10])
eigenvector_centrality = nx.eigenvector_centrality(entity_projection)
print("Top entities by eigenvector centrality:", sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)[:10])

It can be seen in the results that even though different centrality measueres is used there are very little change in the key entities. The differences in the results are caused by the differences in the methods. The methods used is:

Degree Centrality
- Shows the most direct connections to other entities.

Betweenness Centrality
- Shows which entities help as connections to other entities another way of expressing the relationship is that its the entities that lies between other entities the most.

Eigenvector Centrality
- Shows the influence of entities within the network as the most central eg. two, wes craven and elm street often is connected to other central entities.

**Code block 17**


This code examines the clustering and connectivity of the entities. By analyzing the clustering coefficient and average path length, it gains insights into how closely related the entities is to eachther in the network.

In [ ]:
clustering_coefficient = nx.average_clustering(entity_projection)
print("Clustering Coefficient:", clustering_coefficient)
if nx.is_connected(entity_projection):
    avg_path_length = nx.average_shortest_path_length(entity_projection)
    print("Average Path Length:", avg_path_length)
else:
    print("The graph is not connected, so the average path length cannot be calculated.")

The clustering coefficient of 0.658 indicates that approximet 66% of entities connected to a common entity such as character or location also is interconnected. This indicates that only 33% of entities in the reviews of the movie isnt frequently occuring.

The average path length of 2.21 indicates a short average path length. What this means is that the netwrok is highly interconnected. This makes sence since its only one movie thats looked upon and the focus of the reviews is the main story rather than the small details of none important entities.

**Code block 18**

This code creates an interactive and visually dynamic representation of a network graph, showcasing how entities are interconnected. Leveraging Plotly for interactivity and NetworkX for graph structure, it provides an engaging plot where each entity is a node, and connections are formed based on shared associations. The result is a scalable visualization that highlights each entities role and prominence within the network.

By calculating and representing centrality, the plot identifies frequently occuring entities, potentially highlighting recurring themes or influential carracter within the horror narative of the movie. Users can interact with the graph by hovering over nodes to see individual titles and centrality scores, offering a clear, user-friendly way to explore the web of relationships. The scalable nature of the visualization allows for a high-level overview of connectivity, while also enabling detailed examination of specific entities and their connections within the network.

In [ ]:
import plotly.graph_objects as go
import networkx as nx
pos = nx.spring_layout(entity_projection, k=0.3, seed=42)
edge_x = []
edge_y = []
for edge in entity_projection.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
node_sizes = []
node_text = []

for node in entity_projection.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_sizes.append(10 + 50 * degree_centrality[node])  # scale node sizes
    node_text.append(f"{node}<br>Degree Centrality: {degree_centrality[node]:.2f}")

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    text=node_text,
    textposition="top center",
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='Blues',
        size=node_sizes,
        color=node_sizes,
        colorbar=dict(
            title="Degree Centrality",
            thickness=15,
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
                layout=go.Layout(
                    title="Interactive Projection Network",
                    title_x=0.5,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )

fig.show()


This network shows several prominent entities such as Elm Street, Wes Craven and so on which is also shown above.

Each node’s size and color intensity represent its degree centrality, showing the number of direct connections (shared entities or themes) it has with other nodes. Nodes with higher degree centrality are larger and darker, indicating they have more connections or edges in the network.

There is pramarily two clusters in the network that is clusterd around either Elm street or the persons Rachel McAdams or Wes Craven. It is also clear that there is a distinction of persons and what we can assume is whats happening or regarding elm street.

# 3. Text Classification Select and implement one of these approaches:

- LLM-based classification system
- **Few-shot learning implementation using SetFit**
- Traditional NLP classification methods (using existing or synthetic training data)

Classifies the reviews into: Positive, negative or neutral

**Code block 19**

This code prepares to implement SetFit, a library for fine-tuning sentence transformers for classification tasks, ideal for applications where labeled data is limited. By leveraging SetFit’s model and trainer classes, this setup facilitates efficient text classification. The code also includes steps to split the data into training and test sets using train_test_split from scikit-learn.

In [ ]:
!pip install setfit

from setfit import SetFitModel, SetFitTrainer
from sklearn.model_selection import train_test_split
import pandas as pd


**Code block 20**

This code creates a small labeled dataset for few-shot training, providing examples of different sentiments (positive, negative, and neutral) in movie reviews. This data is then structured in a DataFrame, with sentiment labels encoded as integers, setting it up for training a classification model with limited samples.

In [ ]:
import pandas as pd

few_shot_data = [
    {"text": "A terrifying and thrilling movie that kept me on the edge!", "label": "positive"},
    {"text": "The plot was boring and uninspiring.", "label": "negative"},
    {"text": "It was okay, not too scary but had some good moments.", "label": "neutral"}
]
few_shot_df = pd.DataFrame(few_shot_data)
few_shot_df['label'] = few_shot_df['label'].map({"positive": 1, "negative": 0, "neutral": 2})

**Code block 21**

This code fine-tunes a sentence transformer model on a small, labeled dataset for sentiment classification using SetFit. It leverages Hugging Face’s Dataset class and the SetFitTrainer to train the model on few-shot data, making it adaptable for cases with limited labeled examples.

In [ ]:
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer
few_shot_dataset = Dataset.from_pandas(few_shot_df)
model = SetFitModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
trainer = SetFitTrainer(
    model=model,
    train_dataset=few_shot_dataset,
    num_iterations=20
)
trainer.train()

# c671e11fe2b645508e7d3cd9c16d0177525f93cf

**Code block 22**

The code block uses a sentiment analysis model to predict the sentiment of three sample horror movie reviews.

In [ ]:
new_reviews = ["An absolute horror masterpiece!", "The film was a letdown.", "Not scary, but it was entertaining."]
predictions = model(new_reviews)
for review, sentiment in zip(new_reviews, predictions):
    print(f"Review: {review} -> Sentiment: {sentiment}")

# 4. Topic Modeling

1. Leverage LLMs to extract and categorize key themes and topics

2. Apply BERTopic for advanced topic modeling

3. Create clear and insightful visualizations of:
  * Topic distributions
  * Theme relationships
  * Temporal patterns (if applicable)

**Code block 23**

This code prepares the environment and imports necessary libraries for BERTopic—a topic modeling technique that uses transformer embeddings to identify themes within a text dataset. BERTopic leverages dimensionality reduction (UMAP), clustering, and transformer-based embeddings to create interpretable topic clusters, making it effective for organizing large text datasets by themes or topics.

In [ ]:
!pip install bertopic
!pip install umap-learn
!pip install sentence-transformers
!pip install networkx
import re
import networkx as nx
import matplotlib.pyplot as plt
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from umap import UMAP
import numpy as np

**Code block 24**

This line initializes a SentenceTransformer model called 'all-MiniLM-L6-v2', which is specifically designed for efficient text embedding. The model translates each sentence into a numerical embedding that captures its semantic meaning, making it suitable for various NLP tasks like clustering, topic modeling, and similarity analysis.

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

**Code block 25**

This code cleans the reviews of tags and whitespace, then sets a collection of words that is excluded from the creation of topic words. From there a vectorization is made to create uni- and bigrams (1 and 2 word combinations) to enable conversion of text into a format suitable for analysis by enhances feature extraction.

UMAP reduces the dimentions to 2 for plotting and with embedding the numerical representations is made. With BERTopic, the 3 previous parts is used to extract topics based on a minimum of reviews to create them.

In [ ]:
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'\s+', ' ', text)   # Remove extra whitespace
    return text.strip()
reviews = [preprocess_text(review) for review in horror_reviews['text'].dropna().tolist()]
custom_stop_words = [
    "the", "of", "and", "in", "to", "that", "has", "is", "are", "one",
    "it", "this", "with", "for", "as", "on", "by", "an", "at", "from",
    "but", "be", "br", "about", "more", "film", "she", "he", "they",
    "movie", "movies", "you", "not", "all", "was", "so", "his"
]
vectorizer = CountVectorizer(stop_words=custom_stop_words, ngram_range=(1, 2))
umap_model = UMAP(n_neighbors=15, n_components=2, min_dist=0.1, metric='cosine')

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    min_topic_size=2,
    verbose=True
)

topics, probabilities = topic_model.fit_transform(reviews)


**Code block 26**

This code will retrive the model from the previous code block and will print the results of the gathered topics

In [ ]:
topic_info = topic_model.get_topic_info()
print(topic_info)

**In the topic_info table**, each topic is assigned a unique identifier, with four main topics numbered 0 through 3. An additional topic labeled -1 represents "outliers," or reviews that do not strongly align with any specific topic. The "Count" column shows how many reviews fall into each topic; for instance, the outlier topic (-1) includes 7 reviews, while topic 1 contains 6 reviews. Each topic is briefly summarized under "Name" using a few prominent words, providing a quick way to identify its main themes or associated entities. For example, Topic 0 is summarized with words like "freddy," "there," and "nightmare," suggesting a connection to the Nightmare on Elm Street series.

**The "Representation" section** lists key terms associated with each topic, offering a more detailed view of what each topic represents. These terms are the most characteristic for each topic and help define its theme. For example, words like "freddy," "elm street," and "nightmare" in Topic 0 clearly indicate themes related to horror, particularly A Nightmare on Elm Street.

**The "Representative_Docs" section** provides sample reviews that represent each topic, giving a better understanding of the content within each category.

**Code block 27**

The code below generates a heatmap of the produced topics and shows how similar the topics are to eachother

In [ ]:
topic_model.visualize_heatmap().show()

In the similarity matrix there is shown that all topics have a higher similarity score that shows they are similar in some capacity to eachother. This means there will be similar content, language or themes within the topics

The similarity matrix visualizes the relationships between two identified topics based on shared themes or terms, indicating how closely these topics are related to each other.

The color gradient reflects the level of similarity, with darker blue representing higher similarity (closer to 1) and lighter green representing lower similarity (closer to 0.75). Cells along the diagonal, where each topic is compared with itself, display the darkest color, indicating perfect similarity with a score of 1.

For the off-diagonal cells, which show the similarity between different topics, the comparison between Topic 0 and Topic 1 reveals moderate similarity, as shown by a light green shade, suggesting some thematic overlap. This similarity analysis helps illustrate how closely related each topic is in terms of shared elements, highlighting thematic clusters within the dataset.

**Code block 28**

The code organizes topics into a tree-like structure, where similar topics are grouped closer together, showing overarching themes and subtopics.

In [ ]:
topic_model.visualize_hierarchy().show()

Here we see the Hierarchy of the topics, where the topics has highest similarity is paired first. The graph shows a high level of similarity, but since there isnt any other topics there is not more that can be taken form this graph and no comparison between multiple topics can be made

**Code block 29**

This code block performs a manual 2D visualization of topic embeddings using PCA, this is done by reducing the dimensions and therefore simplifing the data.
Afterwards it is plotted to show how the topic lie compared to eachother

In [ ]:
topic_embeddings = topic_model.topic_embeddings_
reduced_embeddings = PCA(n_components=2).fit_transform(topic_embeddings)

plt.figure(figsize=(10, 10))
plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], s=100)
for i, topic in enumerate(topic_info['Name']):
    plt.annotate(topic, (reduced_embeddings[i, 0], reduced_embeddings[i, 1]), fontsize=12)
plt.title("Manual 2D Visualization of Topics")
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.show()

This shows the 2D PCA visualization and the spread of the topics.

The distance between points represents how similar or different the topics are. "0_freddy_there_some_nightmare" is closer to "-1_her_nightmare_nightmare elm_elm street" than to "1_her_would_have_craven," indicating that these two samples may share more content. The distance shows that there is distinction between the different points.

**Code block 30**

This code block constructs and visualizes a network graph that represents relationships among topics based on their cosine similarity and provides a visual representation of how topics are interconnected based on their similarities, helping to identify clusters and relationships within the topic space.

In [ ]:
similarity_matrix = cosine_similarity(topic_embeddings)
threshold = 0.6
G = nx.Graph()

for topic_num, topic_name in enumerate(topic_info['Name']):
    if topic_num != -1:
        G.add_node(topic_num, label=topic_name)

for i in range(len(similarity_matrix)):
    for j in range(i + 1, len(similarity_matrix[i])):
        if similarity_matrix[i][j] > threshold:
            G.add_edge(i, j, weight=similarity_matrix[i][j])

pos = nx.spring_layout(G, k=0.5)
plt.figure(figsize=(10, 10))
nx.draw_networkx(G, pos, with_labels=True, font_size=10, node_size=3000, node_color="skyblue")
nx.draw_networkx_edge_labels(G, pos, edge_labels={(i, j): f"{similarity_matrix[i][j]:.2f}" for i, j in G.edges})
plt.title("Topic Relationship Network")
plt.show()


This shows the simplification of the previous graph, where the distances between the different points is displayed numericly.

The "Topic Relationship Network" graph illustrates the similarity between four identified topics (labeled 0, 1 and 2) based on cosine similarity scores. Each node represents a topic, with edges connecting them to show the degree of similarity, and the scores labeled on each edge.

The edges' thickness and length reflect the strength of similarity. Higher scores, like the 0.84 between topics 0 and 2, indicate a stronger thematic link, suggesting these topics have more shared elements or themes. In contrast, lower scores, such as the 0.74 between topics 1 and 2, imply a weaker relationship, meaning these topics are more distinct.